# PYNQ 튜토리얼 : Xlnk 및 연속 메모리 할당

`Xlnk()`는 연속적인 메모리 블록을 할당 할 수 있다. 메모리 버퍼의 데이터는 PS와 PL간에 효율적으로 전송 될 수 있다.

주의:`Xlnk()`드라이버는 오버레이와 무관하므로 사용하는 오버레이에 관계없이 사용할 수 있다.

## 준비

다음 셀에서 우리는 몇 가지 유용한 함수를 정의 할 것이다.

In [1]:
from time import sleep
from pprint import pprint

def get_kb(mmu):
    return int(mmu.cma_stats()['CMA Memory Available']//1024)

def get_bufcount(mmu):
    return int(memmanager.cma_stats()['Buffer Count'])

def print_kb(mmu):
    print("Available Memory (KB): " + str(get_kb(mmu)))

## 1 단계: Xlnk의 인스턴스 만들기
Xlnk 인스턴스는 이 쳅터 전체에 걸쳐 `mmu` (Memory Management Unit)로 분류된다.

In [2]:
from pynq import Xlnk
xlnk = Xlnk()

## 2 단계: 상태 확인

In [3]:
pprint(xlnk.cma_stats())

{'Buffer Count': 0, 'CMA Memory Available': 73277440, 'CMA Memory Usage': 0}


## 3 단계: 메모리 할당

In [4]:
import numpy as np 
print("Before memory allocation:")
print_kb(xlnk)

buffer = xlnk.cma_array(shape=(10000000,), dtype=np.float32)

print("After memory allocation:")
print_kb(xlnk)

Before memory allocation:
Available Memory (KB): 71560
After memory allocation:
Available Memory (KB): 40584


## 4 단계: 메모리 버퍼 주소 확인

가상 주소는 Linux에서 사용할 수 있다. Physical address는 PL 내의 주변 장치로 전달 될 수 있다.


In [5]:
print("Buffer pointer address (physical memory):")
print(hex(buffer.physical_address))

Buffer pointer address (physical memory):
0x18100000


## 5 단계: 메모리를 비우십시오.
사용 후 연속 메모리를 해제하는 것이 좋다. 이렇게 하면 프로그램에서 메모리가 누출되지 않는다.

In [6]:
del buffer
print(xlnk.cma_stats())

{'CMA Memory Available': 80855040, 'CMA Memory Usage': 0, 'Buffer Count': 0}


In [8]:
print_kb(xlnk)

Available Memory (KB): 79360


주의, 사용 가능한 메모리가 이전 번호와 완전히 같지 않을 수도 있다.